In [90]:
import time
import openpyxl
import pandas as pd
import tkinter as tk
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import requests
import json

def open_credamo(browser):
    browser.get("https://www.credamo.com")
    # 检测登录
    login = False
    for i in range(5):
        try:
            # # show_execute_info("请登录Credamo账号。")
            browser.find_element_by_class_name("login").click()
            time.sleep(1)
            for cookie in browser.get_cookies():
                if cookie['name']=='credamo-dms-auth':
                    login=True
                    break
            # print("未检测到登录，请登录Credamo账号。")
        except:
            time.sleep(1)
            pass
            # print("已登陆。")
    if login==True:
        # show_execute_info("已登陆，请在浏览器中手动打开您需要拒绝被试的项目。")
        print("已登陆，请在浏览器中手动打开您需要拒绝被试的项目。")
    else:
        # show_execute_info("请登录账号，然后打开一个项目。")
        print("已登陆，请在浏览器中手动打开您需要拒绝被试的项目。")
    # show_execute_info("准备好excel后点击自动拒绝被试")
    print("已登陆，请在浏览器中手动打开您需要拒绝被试的项目。")
    
def get_surveyId(browser):
    url = browser.current_url
    try:
        # 获得surveyId
        startloc = url.index("surveyId")+len("surveyId=")
        endloc = url.index("#")
        surveyId = url[url.index("surveyId")+len("surveyId="):url.index("#")]
    except:
        print("还没有打开项目,读取不到surveyId,请打开一个项目")
        # show_execute_info("还没有打开项目,读取不到surveyId,请打开一个项目")
        return 0
    
    #检测是否打开数据清理页面，如果没有则自动点击清理
    try:
        browser.find_element_by_class_name("iconfont icon-shaixuan")
        # 如果找到，表明已经打开数据清理页面
        print("已打开数据清理页面")
        # show_execute_info("已打开数据清理页面")
    except:    
    # 如果报错，没有打开数据清理页面，自动点击“数据清理”
        try:
            browser.find_element_by_class_name("iconfont icon-shaixuan")
        except:
            browser.find_element_by_link_text("数据清理").click()
            time.sleep(3)
    
    return int(surveyId)

def set_page_size(browser,page_size):
    page_sizes = [5,10,30,50,100]
    try:
        size_choise = page_sizes.index(page_size)
    except:
        print("没有该条数/页的选项，请检查")
        # show_execute_info("没有该条数/页的选项，请检查")
        return -2
    # 设置100页
    set_size = False
    retry_time = 0
    while (not set_size) and retry_time <= 5:
        try:
            browser.find_element_by_class_name('el-input__inner').click()
            time.sleep(1)
            pageSizes = browser.find_elements_by_class_name('el-select-dropdown__item')
            pageSizes[size_choise].click()
            page_size = page_sizes[size_choise]
            set_size = True
            return page_size
        except:
            print("设置条数/页失败，正在重试")
            # show_execute_info("设置条数/页失败，正在重试")
            retry_time += 1
    print("设置页码失败，请检查网页。")
    # show_execute_info("设置页码失败，请检查网页。")

def go_to_page(browser,page):
    # try:
    loc = browser.find_element_by_class_name('el-pagination__jump').find_element_by_class_name('el-input__inner')
    loc.send_keys(Keys.CONTROL+'a')     #全选
    loc.send_keys(Keys.DELETE)		# 删除，清空
    loc.send_keys(page)	# 写入新的值
    loc.send_keys(Keys.ENTER)
    return page
    # except:
    #     print("无法跳转页面")
    #     # show_execute_info("无法跳转页面")
    #     return 0

def set_encoding(browser):
    time.sleep(1)
    browser.find_element_by_link_text("数据清理").click()
    time.sleep(2)
    # delete some click that cause bugs 20231210
    # browser.find_element_by_xpath("/html/body/div[@id='survey_bg']/div[4]/div[1]/div[@class='route']/div[@class='button2']").click()
    # time.sleep(1)
    # browser.find_element_by_xpath("/html/body/div[@id='survey_bg']/div[4]/div[@class='resultView']/div[@class='search-box']/div[@class='popover'][4]").click()
    # time.sleep(1)
    # browser.find_element_by_xpath("/html/body/div[@id='survey_bg']/div[4]/div[1]/div[@class='route']/div[@class='button1']").click()
    # time.sleep(1)

def get_page_df(browser,surveyId,page_size,page_num,download=False):
    try:
        time.sleep(1)
        cookies = browser.get_cookies()
        cookie_str = ""
        for cookie in cookies:
            cookie_str = "{}={}; ".format(cookie['name'],cookie['value']) + cookie_str
        cookie_str = cookie_str[:-2]
        
        headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
                    "referer": "https://www.credamo.com/survey.html?surveyId=%d}" % surveyId,
                    "cookie":cookie_str}
        url = "https://www.credamo.com/v1/cleanVar/qstOverviewBySurId/%d?currPageSize=%d&currPageIndex=%d"  % (surveyId,page_size,page_num)
    except:
        print("获取cookie失败，请重试。")
        # show_execute_info("获取cookie失败，请重试。")
        return []
    try:
        resp = requests.get(url=url,headers=headers)
        if resp.status_code == 200:
            output = json.loads(resp.content.decode('utf8'))
            data = output['data']
            ques_headers = data['header']
            total_cnt = output['total']
            ques_headers_kv = {}
            for header in ques_headers:
                ques_headers_kv[header["id"]] = header['qNum']+" "+header['questionName']
            df = pd.DataFrame(output['data']['rowList'])
            #如果下载，则不替换列名称，最后再替换
            if not download:
                df.columns = [ques_headers_kv[id] if id not in ['status', 'answerId', 'userId'] else id for id in df.columns]
                return df,total_cnt
            else:
                return df,total_cnt,ques_headers_kv,ques_headers
        else:
            print("无法登录到问卷页面")
            # show_execute_info("无法登录到问卷页面")
            return pd.DataFrame([]),0
    except:
        print("无法get问卷页面")
        # show_execute_info("无法get问卷页面")
        return pd.DataFrame([]),0

def download_data(browser,download_name):
    surveyId = get_surveyId(browser)
    set_encoding(browser)
    page_size = set_page_size(browser,page_size=10)
    page_num = 1
    page_num = go_to_page(browser,page_num)
    page_df,total_cnt,ques_headers_kv,ques_headers = get_page_df(browser,surveyId,page_size,page_num,download=True)
    total_page_num = total_cnt//page_size+1
    page_dfs = pd.DataFrame([])
    for page_num in range(total_page_num):
        page_num += 1
        print(page_num)
        # go_to_page(browser,page_num)
        # time.sleep(5)
        page_df,total_cnt,ques_headers_kv,_ = get_page_df(browser,surveyId,page_size,page_num,download=True)
        if 'userId' in page_df.columns:
            page_df.set_index('userId')
            if str(page_dfs.shape)=="(0, 0)":
                page_dfs = page_df
            else:
                page_dfs = pd.concat([page_dfs,page_df],ignore_index=True)
    page_dfs.columns = [
        ques_headers_kv[id] 
        if id not in ['status', 'answerId', 'userId'] 
        else id 
        for id in page_dfs.columns
    ]
    page_dfs = page_dfs[['status', 'answerId', 'userId'] + [ques_headers_kv[ques_header['id']] for ques_header in ques_headers]]
    page_dfs = page_dfs[(page_dfs=="*").sum(axis=1)==0] #剔除拒绝的
    download_name_str = download_name.get('0.0','end').strip()+"_"+time.strftime('%Y%m%d_%H%M')+".xlsx"
    page_dfs.to_excel(download_name_str)
    # show_execute_info("{}下载完成".format(download_name_str))

# def get_userId_loc(excel_name,page_df,page_num,autobatchReject):    
#     chosen_ids = excel_name.get("1.0","end").strip().split('\n')
#     print('指定拒绝的被试:',chosen_ids)
    
#     page_userIds = page_df['userId'].tolist()
#     print(page_df.shape,page_df.columns)
#     print('page_userIds',page_userIds)
#     page_chosen_ids = list(set(chosen_ids) & set(page_userIds))
#     print('page_chosen_ids')
#     chosen_rows = []
#     for chosen_id in page_chosen_ids:
#         print('page_userIds',page_userIds)
#         chosen_row = page_userIds.index(chosen_id)+1
#         chosen_rows.append(chosen_row)
#         print('chosen_row:', chosen_row)
#         # change to relative xpath 20231227
#         xpath_expression = f"//table[@style='margin-top: 0px; width: 415px;']/tbody/tr[{chosen_row}]/td[1]/div/span"
#         browser.find_element_by_xpath(xpath_expression).click()
#     # 批量拒绝
#     browser.find_element_by_class_name("el-dropdown").click() # 点击批量操作
#     time.sleep(1)
#     if autobatchReject == True:
#         browser.find_elements_by_class_name("el-dropdown-menu__item")[1].click() # 点击拒绝选中数据
#         time.sleep(1)
#         el_select = browser.find_elements_by_class_name("el-select")[1].click()
#         msgbox = browser.find_element_by_class_name("el-message-box")
#         msgbox.click()
#         print("请根据网页提示进行操作")
#         time.sleep(3)
#         msgbox.find_element_by_class_name("el-button").click()
#         print("第%d页已拒绝%d个被试" % (page_num,len(chosen_rows)))
#         # show_execute_info("第%d页已拒绝%d个被试" % (page_num,len(chosen_rows)))
#         time.sleep(3)
#     return chosen_rows 
def get_userId_loc(excel_name,page_df,page_num,autobatchReject):
    chosen_ids = excel_name.get("1.0","end").strip().split('\n')
    
    page_userIds = page_df['userId'].tolist()
    page_chosen_ids = list(set(chosen_ids) & set(page_userIds))
    chosen_rows = []
    for chosen_id in page_chosen_ids:
        chosen_row = page_userIds.index(chosen_id)+1
        chosen_rows.append(chosen_row)
        xpath_expression = f"//table[@style='margin-top: 0px; width: 415px;']/tbody/tr[{chosen_row}]/td[1]/div/span"
        browser.find_element_by_xpath(xpath_expression).click()
        # 批量拒绝
    browser.find_element_by_class_name("el-dropdown").click()
    time.sleep(1)
    if autobatchReject == True:
            browser.find_elements_by_class_name("el-dropdown-menu__item")[1].click() # 点击拒绝选中数据
            time.sleep(3)
            try:
                WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='el-select' and @style='width: 380px;']"))).click()  # 点击请选择
            except TimeoutException:
                ok_button_locator = (By.XPATH, "//button[@class='el-button el-button--default el-button--small el-button--primary ']/span[contains(text(), '确定')]")
                WebDriverWait(browser, 10).until(EC.element_to_be_clickable(ok_button_locator)).click()
                # show_execute_info("第%d页无拒绝被试" % page_num)
                print("第%d页无拒绝被试" % page_num)
                time.sleep(3)
                return []
            reject_reason_li_locator = (By.XPATH, "//li[contains(@class, 'el-select-dropdown__item') and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]")
            WebDriverWait(browser, 10).until(EC.element_to_be_clickable(reject_reason_li_locator)).click()
            # 等待第一个按钮的操作完成，例如等待下拉框消失
            # WebDriverWait(browser, 10).until_not(EC.presence_of_element_located((By.XPATH, reject_reason_li_locator)))
            reject_button_locator = (By.XPATH, "//div[@class='el-dialog__footer']//button[@class='el-button el-button--primary' and span[text()='批量拒绝']]")
            WebDriverWait(browser, 10).until(EC.element_to_be_clickable(reject_button_locator)).click()
            print("第%d页已拒绝%d个被试" % (page_num,len(chosen_rows)))
            time.sleep(3)
    return chosen_rows 


def batchReject(browser,excel_name,page_num_assign_text,autoReject=True):
    surveyId = get_surveyId(browser)
    page_size = set_page_size(browser,page_size=10)
    page_num = 1
    page_num = go_to_page(browser,page_num)
    page_df,total_cnt = get_page_df(browser,surveyId,page_size,page_num)
    total_page_num = total_cnt//page_size+1
    if autoReject:
        for page_num in range(total_page_num):
            page_num += 1
            # print(page_num)
            go_to_page(browser,page_num)
            time.sleep(5)
            page_df,total_cnt = get_page_df(browser,surveyId,page_size,page_num)
            # print(total_cnt)
            userId_loc = get_userId_loc(excel_name,page_df,page_num,autoReject)
            # print(userId_loc)
        # show_execute_info('所有页面已拒绝。\n你可以打开另一个项目，清空原用户ID后粘贴新项目待拒绝用户ID，重新点击拒绝被试。\n或者关闭浏览器，关闭本程序')
    else:
        #手动拒绝
        try:
            page_num_assign = int(page_num_assign_text.get('0.0','end').strip())
        except:
            print("无法读取页面数，请在上方小方框中输入正确的页面数字")
            # show_execute_info("无法读取页面数，请在上方小方框中输入正确的页面数字")
            return 0
        if page_num_assign <= 0 or page_num_assign > total_cnt//page_size+1:
            print("输入的页面数字超过答卷范围，请检查页面数")
            # show_execute_info("输入的页面数字超过答卷范围，请检查页面数")
        go_to_page(browser,page_num_assign)
        time.sleep(5)
        page_df,total_cnt = get_page_df(browser,surveyId,page_size,page_num_assign)
        # print(total_cnt)
        userId_loc = get_userId_loc(excel_name,page_df,page_num_assign,autoReject)
        # print(userId_loc)
        print('页面%d已选中待拒绝被试，可以手动在浏览器中点击拒绝。\n' % page_num_assign)
        # show_execute_info('页面%d已选中待拒绝被试，可以手动在浏览器中点击拒绝。\n'%page_num_assign)

# def show_execute_info(var,cmd=False):
#     if cmd:
#         print(var)
#     else:
#         execute_info.insert("0.0", '\n' + time.strftime('%Y-%m-%d %H:%M:%S ') + var + '\n')

In [88]:
browser = webdriver.Chrome()

In [89]:
open_credamo(browser)

已登陆，请在浏览器中手动打开您需要拒绝被试的项目。
已登陆，请在浏览器中手动打开您需要拒绝被试的项目。


In [91]:
window = tk.Tk()
excel_name = tk.Text(window, height=10)
excel_name.pack()

In [92]:
#
excel_name.insert("0.0", 
"""
2MdZobzL
o7jnjvB7
9LQZdpAe
0eyrZWGM
qM6oBy07
5M2bDdNM
KLOZz617
yev1DAXL
qM6zRbxM
J7BoqmdL
qM6o1nJ7
2MdZQBZL
kMrbGQVM
AM9z9zk7
DMxao357
JekwpEYe
2MdZ85zL
z7ljnk2e
xMo8GzK7
WMRZjDPM
9LQR3Wve
pLz5ED57
DMxr1YKL
leNZP6ge
QL0o9a07
2MdZbwWL
AM9DPVne
oe4oGrWL
G7XZQQde
RMWZxmbe
aMg1vAnL
o7jr40l7
qM6ogNd7
kMrKEVk7
pLz5w3K7
aMg1xOyL
kM1oA2zL
xLVZ0xRL
Jek8mvQM
p7YB9jWL
9LQlg0Q7
Ke5o0mV7
5M2b5BkM
nePnYry7
yevD5Eb7
veEBrqJM
5LwjN517
8MZm14Ke
5LwjRE27
J7BomydL
27aDlYOM
z7AoV9Pe
5M2bQE8M
DMxXnKJM
leNZx6Re
QL0zaaJM
leNZdqme
kM1oPQQL
AM9D6ARe
5LwjX8V7
RMWZ85re
WMRZjEdM
AM9D3YVe
oe4o69qL
2MdZ926L
27aRV0VL
qM6oPm87
5LwjqaY7
4enyENdL
nePZNXy7
me3ozkbe
2MdZb46L
xLV2WrVL
27akK0Ve
WLGonDxL
RMWZ54Ge
VMJZ6JwM
G7XoAQBM
G7XQEnGL
xebw9GNM
5M2b6GVM
G7XZPZ5e
G7XQn65L
z7A3VyXe
WMRZzAdM
RMWxPvE7
yevKmDNM
0eyaAG9e
G7XvO1Be
5LmEjNOL
xLVZNaVL
9LQoBJ4M
WMRj8B9M
kMrK5Q67
0eyl92VL
JekoyPYe
5LwjXpn7
0eyl0wGL
2Md4XyXM
9LQZBgQe
z7lkmVpM
ve8o9wrL
27akoPbe
4eny1E0L
WMRZ1ZdM
4enl9rB7
5LwjqVV7
ve8okdWL
G7XZQEGe
5M2bdoNM
J7qgnjqe
AM9D0aWe
qM6rGyRM
9LQZQ8Be
JekwRGwe
ve8z0bz7
z7AovkPe
BepqKXBM
4enyvEAL
J7Bo3b1L
""")

In [54]:
excel_name.get("1.0","end").strip().split('\n')

['2MdZobzL',
 'o7jnjvB7',
 '9LQZdpAe',
 '0eyrZWGM',
 'qM6oBy07',
 '5M2bDdNM',
 'KLOZz617',
 'yev1DAXL',
 'qM6zRbxM',
 'J7BoqmdL',
 'qM6o1nJ7',
 '2MdZQBZL',
 'kMrbGQVM',
 'AM9z9zk7',
 'DMxao357',
 'JekwpEYe',
 '2MdZ85zL',
 'z7ljnk2e',
 'xMo8GzK7',
 'WMRZjDPM',
 '9LQR3Wve',
 'pLz5ED57',
 'DMxr1YKL',
 'leNZP6ge',
 'QL0o9a07',
 '2MdZbwWL',
 'AM9DPVne',
 'oe4oGrWL',
 'G7XZQQde',
 'RMWZxmbe',
 'aMg1vAnL',
 'o7jr40l7',
 'qM6ogNd7',
 'kMrKEVk7',
 'pLz5w3K7',
 'aMg1xOyL',
 'kM1oA2zL',
 'xLVZ0xRL',
 'Jek8mvQM',
 'p7YB9jWL',
 '9LQlg0Q7',
 'Ke5o0mV7',
 '5M2b5BkM',
 'nePnYry7',
 'yevD5Eb7',
 'veEBrqJM',
 '5LwjN517',
 '8MZm14Ke',
 '5LwjRE27',
 'J7BomydL',
 '27aDlYOM',
 'z7AoV9Pe',
 '5M2bQE8M',
 'DMxXnKJM',
 'leNZx6Re',
 'QL0zaaJM',
 'leNZdqme',
 'kM1oPQQL',
 'AM9D6ARe',
 '5LwjX8V7',
 'RMWZ85re',
 'WMRZjEdM',
 'AM9D3YVe',
 'oe4o69qL',
 '2MdZ926L',
 '27aRV0VL',
 'qM6oPm87',
 '5LwjqaY7',
 '4enyENdL',
 'nePZNXy7',
 'me3ozkbe',
 '2MdZb46L',
 'xLV2WrVL',
 '27akK0Ve',
 'WLGonDxL',
 'RMWZ54Ge',
 'VMJZ6JwM',

In [94]:
def get_accept_userId_loc(excel_name,page_df,page_num,autobatchReject):
    page_userIds = page_df['userId'].tolist()
    reject_ids = excel_name.get("1.0","end").strip().split('\n')
    page_unselected_ids = page_df[page_df['status']==1]['userId'].tolist() # list(set(chosen_ids) & set(page_userIds))
    page_chosen_ids = list(set(page_unselected_ids) - set(reject_ids))
    chosen_rows = []
    for chosen_id in page_chosen_ids:
        chosen_row = page_userIds.index(chosen_id)+1
        chosen_rows.append(chosen_row)
        xpath_expression = f"//table[@style='margin-top: 0px; width: 415px;']/tbody/tr[{chosen_row}]/td[1]/div/span"
        browser.find_element_by_xpath(xpath_expression).click()

    browser.find_element_by_class_name("el-dropdown").click()
    time.sleep(1)
    if autobatchReject == True:
        browser.find_elements_by_class_name("el-dropdown-menu__item")[0].click() # 点击采纳选中数据
        time.sleep(1)
        confirm_result_button_xpath = f"//span[contains(text(), '确定')]"#
        confirm_result_buttons = browser.find_elements_by_xpath(confirm_result_button_xpath)
        # 遍历所有匹配的元素
        for button in confirm_result_buttons:
            # 判断元素是否可点击
            if button.is_enabled() and button.is_displayed():
                # 执行点击操作
                button.click()
                # 只点击第一个可点击的元素，如果需要点击所有可点击的元素，可以去掉break语句
                # print("第%d页已接受%d个被试" % (page_num,len(chosen_rows)))
                # show_execute_info("第%d页已接受%d个被试" % (page_num,len(chosen_rows)))
                print("第%d页已接受%d个被试" % (page_num,len(chosen_rows)))
                break
    else:
        # show_execute_info('页面%d已选中待接受被试，可以手动在浏览器中点击接受。\n'%page_num)
        print("页面%d已选中%d个待接受被试，可以手动在浏览器中点击接受。\n" % (page_num,len(chosen_rows)))
        # show_execute_info("页面%d已选中%d个待接受被试，可以手动在浏览器中点击接受。\n" % (page_num,len(chosen_rows)))
    return 

def batchAccept(browser,excel_name,page_num_assign_text,autoReject=True):
    surveyId = get_surveyId(browser)
    page_size = set_page_size(browser,page_size=10)
    page_num = 1
    page_num = go_to_page(browser,page_num)
    page_df,total_cnt = get_page_df(browser,surveyId,page_size,page_num)
    total_page_num = total_cnt//page_size+1
    if autoReject:
        for page_num in range(total_page_num):
            page_num += 1
            # print(page_num)
            go_to_page(browser,page_num)
            time.sleep(5)
            page_df,total_cnt = get_page_df(browser,surveyId,page_size,page_num)
            # print(total_cnt)
            get_accept_userId_loc(excel_name,page_df,page_num,autoReject)
            # print(userId_loc)
        print('所有页面已接受。\n你可以打开另一个项目，清空原用户ID后粘贴新项目待拒绝用户ID，重新点击拒绝被试。\n或者关闭浏览器，关闭本程序')
        # show_execute_info('所有页面已拒绝。\n你可以打开另一个项目，清空原用户ID后粘贴新项目待拒绝用户ID，重新点击拒绝被试。\n或者关闭浏览器，关闭本程序')
    else:
        #手动拒绝
        try:
            page_num_assign = int(page_num_assign_text.get('0.0','end').strip())
        except:
            print("无法读取页面数，请在上方小方框中输入正确的页面数字")
            # show_execute_info("无法读取页面数，请在上方小方框中输入正确的页面数字")
            return 0
        if page_num_assign <= 0 or page_num_assign > total_cnt//page_size+1:
            print("输入的页面数字超过答卷范围，请检查页面数")
            # show_execute_info("输入的页面数字超过答卷范围，请检查页面数")
        go_to_page(browser,page_num_assign)
        time.sleep(5)
        page_df,total_cnt = get_page_df(browser,surveyId,page_size,page_num_assign)
        # print(total_cnt)
        get_accept_userId_loc(excel_name,page_df,page_num_assign,autoReject)
        # print(userId_loc)
        print('页面%d已选中待接受被试，可以手动在浏览器中点击接受。\n' % page_num_assign)
        # show_execute_info('页面%d已选中待拒绝被试，可以手动在浏览器中点击拒绝。\n'%page_num_assign)

In [114]:
def go_to_page(browser,page):
    # try:
    loc = WebDriverWait(browser.find_element_by_class_name('el-pagination__jump'), 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'el-input__inner'))
    )
    loc.send_keys(Keys.CONTROL+'a')     #全选
    loc.send_keys(Keys.DELETE)		# 删除，清空
    loc.send_keys(page)	# 写入新的值
    loc.send_keys(Keys.ENTER)
    return page

In [109]:
browser.find_element_by_class_name('el-pagination__jump').find_elements_by_class_name('el-input__inner')

[<selenium.webdriver.remote.webelement.WebElement (session="4d606c57b0336c7de1f464ef70842ea3", element="37d40c95-246b-4d9d-a740-0680c1abd175")>]

In [112]:
loc = WebDriverWait(browser.find_element_by_class_name('el-pagination__jump'), 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'el-input__inner'))
)
loc.send_keys(Keys.CONTROL+'a')     #全选

In [115]:
page_num_assign_text = tk.Text(window, height=1, width=4)
page_num_assign_text.insert("0.0", '1')

batchAccept(
    browser=browser,
    excel_name=excel_name,
    page_num_assign_text=page_num_assign_text,
    autoReject=True
)

第1页已接受0个被试
第2页已接受0个被试
第3页已接受0个被试
第4页已接受0个被试
第5页已接受5个被试
第6页已接受6个被试


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=109.0.5414.120)


In [107]:
loc = browser.find_element_by_class_name('el-pagination__jump').find_element_by_class_name('el-input__inner')
loc.send_keys(Keys.CONTROL+'a')     #全选
# loc.send_keys(Keys.DELETE)		# 删除，清空
# loc.send_keys(page)	# 写入新的值
# loc.send_keys(Keys.ENTER)

In [60]:
select_all_checkbox_locator = (By.XPATH, "//table[@style='width: 415px;']//span[@title='全选/取消']")
WebDriverWait(browser, 10).until(EC.element_to_be_clickable(select_all_checkbox_locator)).click()

TimeoutException: Message: 


In [71]:
xpath_expression = f"//span[@title='全选/取消' and @class='vxe-cell--checkbox']"#
browser.find_elements_by_xpath(xpath_expression)[1].click()

In [105]:
browser.find_element_by_class_name("el-dropdown").click()

In [106]:
browser.find_elements_by_class_name("el-dropdown-menu__item")[0].click() # 点击采纳选中数据
xpath_expression = f"//span[contains(text(), '确定')]"#
confirm_result_buttons = browser.find_elements_by_xpath(xpath_expression)
# 遍历所有匹配的元素
for button in confirm_result_buttons:
    # 判断元素是否可点击
    if button.is_enabled() and button.is_displayed():
        # 执行点击操作
        button.click()
        # 只点击第一个可点击的元素，如果需要点击所有可点击的元素，可以去掉break语句
        break

In [25]:
ok_button_locator = (By.XPATH, "//button[@class='el-button el-button--primary']/span[contains(text(), '确定')]")
WebDriverWait(browser, 10).until(EC.element_to_be_clickable(ok_button_locator)).click()

TimeoutException: Message: 


In [23]:
page_num_assign_text = tk.Text(window, height=1, width=4)
page_num_assign_text.insert("0.0", '14')

batchAccept(
    browser=browser,
    excel_name=excel_name,
    page_num_assign_text=page_num_assign_text,
    autoReject=False
)

页面14已选中待拒绝被试，可以手动在浏览器中点击拒绝。



In [7]:
batchReject(
    browser=browser,
    excel_name=excel_name,
    page_num_assign_text=1,
    autoReject=True
)

第1页无拒绝被试
第2页无拒绝被试
第3页无拒绝被试
第4页无拒绝被试
第5页无拒绝被试
第6页无拒绝被试
第7页无拒绝被试
第8页无拒绝被试
第9页无拒绝被试
第10页无拒绝被试
第11页无拒绝被试
第12页无拒绝被试
第13页无拒绝被试
第14页无拒绝被试
第15页无拒绝被试
第16页无拒绝被试
第17页无拒绝被试
第18页无拒绝被试
第19页无拒绝被试
第20页无拒绝被试
第21页无拒绝被试
第22页已拒绝4个被试
第23页已拒绝3个被试
第24页已拒绝4个被试
第25页已拒绝2个被试
第26页已拒绝2个被试
第27页已拒绝2个被试
第28页已拒绝2个被试
第29页已拒绝2个被试
第30页已拒绝2个被试


KeyError: 'userId'

In [21]:
raw_string = r"\u001b[1;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[1;31mKeyError\u001b[0m                                  Traceback (most recent call last)\n\u001b[1;32mc:\\Users\\k8723\\Anaconda3\\envs\\py36\\lib\\site-packages\\pandas\\core\\indexes\\base.py\u001b[0m in \u001b[0;36mget_loc\u001b[1;34m(self, key, method, tolerance)\u001b[0m\n\u001b[0;32m   2897\u001b[0m             \u001b[1;32mtry\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m-> 2898\u001b[1;33m                 \u001b[1;32mreturn\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0m_engine\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mget_loc\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mcasted_key\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m   2899\u001b[0m             \u001b[1;32mexcept\u001b[0m \u001b[0mKeyError\u001b[0m \u001b[1;32mas\u001b[0m \u001b[0merr\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\n\u001b[1;32mpandas\\_libs\\index.pyx\u001b[0m in \u001b[0;36mpandas._libs.index.IndexEngine.get_loc\u001b[1;34m()\u001b[0m\n\n\u001b[1;32mpandas\\_libs\\index.pyx\u001b[0m in \u001b[0;36mpandas._libs.index.IndexEngine.get_loc\u001b[1;34m()\u001b[0m\n\n\u001b[1;32mpandas\\_libs\\hashtable_class_helper.pxi\u001b[0m in \u001b[0;36mpandas._libs.hashtable.PyObjectHashTable.get_item\u001b[1;34m()\u001b[0m\n\n\u001b[1;32mpandas\\_libs\\hashtable_class_helper.pxi\u001b[0m in \u001b[0;36mpandas._libs.hashtable.PyObjectHashTable.get_item\u001b[1;34m()\u001b[0m\n\n\u001b[1;31mKeyError\u001b[0m: 'userId'\n\nThe above exception was the direct cause of the following exception:\n\n\u001b[1;31mKeyError\u001b[0m                                  Traceback (most recent call last)\n\u001b[1;32m<ipython-input-7-a284706ca00e>\u001b[0m in \u001b[0;36m<module>\u001b[1;34m\u001b[0m\n\u001b[0;32m      3\u001b[0m     \u001b[0mexcel_name\u001b[0m\u001b[1;33m=\u001b[0m\u001b[0mexcel_name\u001b[0m\u001b[1;33m,\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m      4\u001b[0m     \u001b[0mpage_num_assign_text\u001b[0m\u001b[1;33m=\u001b[0m\u001b[1;36m1\u001b[0m\u001b[1;33m,\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m----> 5\u001b[1;33m     \u001b[0mautoReject\u001b[0m\u001b[1;33m=\u001b[0m\u001b[1;32mTrue\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m      6\u001b[0m )\n\n\u001b[1;32m<ipython-input-1-3d4ad3da94c2>\u001b[0m in \u001b[0;36mbatchReject\u001b[1;34m(browser, excel_name, page_num_assign_text, autoReject)\u001b[0m\n\u001b[0;32m    281\u001b[0m             \u001b[0mpage_df\u001b[0m\u001b[1;33m,\u001b[0m\u001b[0mtotal_cnt\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mget_page_df\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mbrowser\u001b[0m\u001b[1;33m,\u001b[0m\u001b[0msurveyId\u001b[0m\u001b[1;33m,\u001b[0m\u001b[0mpage_size\u001b[0m\u001b[1;33m,\u001b[0m\u001b[0mpage_num\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m    282\u001b[0m             \u001b[1;31m# print(total_cnt)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m--> 283\u001b[1;33m             \u001b[0muserId_loc\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mget_userId_loc\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mexcel_name\u001b[0m\u001b[1;33m,\u001b[0m\u001b[0mpage_df\u001b[0m\u001b[1;33m,\u001b[0m\u001b[0mpage_num\u001b[0m\u001b[1;33m,\u001b[0m\u001b[0mautoReject\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m    284\u001b[0m             \u001b[1;31m# print(userId_loc)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m    285\u001b[0m         \u001b[1;31m# show_execute_info('所有页面已拒绝。\\n你可以打开另一个项目，清空原用户ID后粘贴新项目待拒绝用户ID，重新点击拒绝被试。\\n或者关闭浏览器，关闭本程序')\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\n\u001b[1;32m<ipython-input-1-3d4ad3da94c2>\u001b[0m in \u001b[0;36mget_userId_loc\u001b[1;34m(excel_name, page_df, page_num, autobatchReject)\u001b[0m\n\u001b[0;32m    232\u001b[0m     \u001b[0mchosen_ids\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mexcel_name\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mget\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;34m\"1.0\"\u001b[0m\u001b[1;33m,\u001b[0m\u001b[1;34m\"end\"\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mstrip\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0msplit\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;34m'\\n'\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m    233\u001b[0m \u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m--> 234\u001b[1;33m     \u001b[0mpage_userIds\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mpage_df\u001b[0m\u001b[1;33m[\u001b[0m\u001b[1;34m'userId'\u001b[0m\u001b[1;33m]\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mtolist\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m    235\u001b[0m     \u001b[0mpage_chosen_ids\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mlist\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mset\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mchosen_ids\u001b[0m\u001b[1;33m)\u001b[0m \u001b[1;33m&\u001b[0m \u001b[0mset\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mpage_userIds\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m    236\u001b[0m     \u001b[0mchosen_rows\u001b[0m \u001b[1;33m=\u001b[0m \u001b[1;33m[\u001b[0m\u001b[1;33m]\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\n\u001b[1;32mc:\\Users\\k8723\\Anaconda3\\envs\\py36\\lib\\site-packages\\pandas\\core\\frame.py\u001b[0m in \u001b[0;36m__getitem__\u001b[1;34m(self, key)\u001b[0m\n\u001b[0;32m   2904\u001b[0m             \u001b[1;32mif\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mcolumns\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mnlevels\u001b[0m \u001b[1;33m>\u001b[0m \u001b[1;36m1\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   2905\u001b[0m                 \u001b[1;32mreturn\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0m_getitem_multilevel\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mkey\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m-> 2906\u001b[1;33m             \u001b[0mindexer\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mcolumns\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mget_loc\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mkey\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m   2907\u001b[0m             \u001b[1;32mif\u001b[0m \u001b[0mis_integer\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mindexer\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   2908\u001b[0m                 \u001b[0mindexer\u001b[0m \u001b[1;33m=\u001b[0m \u001b[1;33m[\u001b[0m\u001b[0mindexer\u001b[0m\u001b[1;33m]\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\n\u001b[1;32mc:\\Users\\k8723\\Anaconda3\\envs\\py36\\lib\\site-packages\\pandas\\core\\indexes\\base.py\u001b[0m in \u001b[0;36mget_loc\u001b[1;34m(self, key, method, tolerance)\u001b[0m\n\u001b[0;32m   2898\u001b[0m                 \u001b[1;32mreturn\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0m_engine\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mget_loc\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mcasted_key\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   2899\u001b[0m             \u001b[1;32mexcept\u001b[0m \u001b[0mKeyError\u001b[0m \u001b[1;32mas\u001b[0m \u001b[0merr\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m-> 2900\u001b[1;33m                 \u001b[1;32mraise\u001b[0m \u001b[0mKeyError\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mkey\u001b[0m\u001b[1;33m)\u001b[0m \u001b[1;32mfrom\u001b[0m \u001b[0merr\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m   2901\u001b[0m \u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   2902\u001b[0m         \u001b[1;32mif\u001b[0m \u001b[0mtolerance\u001b[0m \u001b[1;32mis\u001b[0m \u001b[1;32mnot\u001b[0m \u001b[1;32mNone\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\n\u001b[1;31mKeyError\u001b[0m: 'userId'"
decoded_string = bytes(raw_string, 'utf-8').decode('unicode-escape')


In [22]:
decoded_string

'\x1b---------------------------------------------------------------------------\x1b\n\x1bKeyError\x1b                                  Traceback (most recent call last)\n\x1bc:\\Users\\k8723\\Anaconda3\\envs\\py36\\lib\\site-packages\\pandas\\core\\indexes\\base.py\x1b in \x1bget_loc\x1b(self, key, method, tolerance)\x1b\n\x1b   2897\x1b             \x1btry\x1b\x1b:\x1b\x1b\x1b\x1b\x1b\x1b\n\x1b-> 2898\x1b                 \x1breturn\x1b \x1bself\x1b\x1b.\x1b\x1b_engine\x1b\x1b.\x1b\x1bget_loc\x1b\x1b(\x1b\x1bcasted_key\x1b\x1b)\x1b\x1b\x1b\x1b\x1b\x1b\n\x1b\x1b   2899\x1b             \x1bexcept\x1b \x1bKeyError\x1b \x1bas\x1b \x1berr\x1b\x1b:\x1b\x1b\x1b\x1b\x1b\x1b\n\n\x1bpandas\\_libs\\index.pyx\x1b in \x1bpandas._libs.index.IndexEngine.get_loc\x1b()\x1b\n\n\x1bpandas\\_libs\\index.pyx\x1b in \x1bpandas._libs.index.IndexEngine.get_loc\x1b()\x1b\n\n\x1bpandas\\_libs\\hashtable_class_helper.pxi\x1b in \x1bpandas._libs.hashtable.PyObjectHashTable.get_item\x1b()\x1b\n\n\x1bpandas\\_libs

In [41]:
el_select = browser.find_element_by_xpath("//div[@class='el-select' and @style='width: 380px;']") # 点击请选择
el_select.click()

In [15]:
reject_reason_li = browser.find_element_by_xpath("//li[@class='el-select-dropdown__item' and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]")
reject_reason_li

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[@class='el-select-dropdown__item' and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]"}
  (Session info: chrome=109.0.5414.120)


In [23]:
reject_reason_li = browser.find_element_by_xpath("//li[contains(@class, 'el-select-dropdown__item') and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]")
reject_reason_li.click()

In [45]:
len(reject_reasons)

10

In [19]:

dropdown__items = browser.find_elements_by_class_name("el-select-dropdown__item")

In [22]:
dropdown__item[-1].click()

In [49]:
reject_button = browser.find_elements_by_xpath("button[@class='el-button el-button--primary' and span[text()='批量拒绝']]")
reject_button

[]

In [50]:
reject_button = browser.find_element_by_xpath("//div[@class='el-dialog__footer']//button[@class='el-button el-button--primary' and span[text()='批量拒绝']]")
reject_button

<selenium.webdriver.remote.webelement.WebElement (session="93ed257b692f814b704b82fdb1c162f2", element="212a5120-41f8-4d60-96d3-bf275ed1e5d1")>

In [23]:
ok_button = browser.find_element_by_xpath("//button[@class='el-button el-button--default el-button--small el-button--primary ']/span[contains(text(), '确定')]")
ok_button.click()

In [ ]:
<div role="dialog" aria-modal="true" aria-label="拒绝问卷" class="el-dialog refuse_dialog_bg" style="margin-top: 15vh; width: 500px;"><div class="el-dialog__header"><span class="el-dialog__title">拒绝问卷</span><button type="button" aria-label="Close" class="el-dialog__headerbtn"><i class="el-dialog__close el-icon el-icon-close"></i></button></div><div class="el-dialog__body"><div data-v-3e6f8ccc="" class="refuse_form"><div data-v-3e6f8ccc="" style="width: 70px;">拒绝原因: </div> <div data-v-3e6f8ccc="" class="new_css"><div data-v-3e6f8ccc="" class="el-select" style="width: 380px;"><!----><div class="el-input el-input--suffix"><!----><input type="text" readonly="readonly" autocomplete="off" placeholder="请选择" class="el-input__inner"><!----><span class="el-input__suffix"><span class="el-input__suffix-inner"><i class="el-select__caret el-input__icon el-icon-arrow-up"></i><!----><!----><!----><!----><!----></span><!----></span><!----><!----></div></div></div></div> </div><div class="el-dialog__footer"><span data-v-3e6f8ccc="" class="dialog-footer"><button data-v-3e6f8ccc="" type="button" class="el-button el-button--default"><!----><!----><span>取消</span></button> <!----> <button data-v-3e6f8ccc="" type="button" class="el-button el-button--primary"><!----><!----><span>批量拒绝</span></button></span></div></div>

In [ ]:
        reject_reason_li_locator = (By.XPATH, "//li[@class='el-select-dropdown__item' and span[text()='填写内容不符合要求，请认真阅读并仔细填答']]")
        WebDriverWait(browser, 20).until(EC.element_to_be_clickable(reject_reason_li_locator)).click()